# Rede Neural Para Classificar Pestinhas

Usando a analogia de que uma rede neural é como uma cozinha gigante comandada por um chef, criaremos uma rede neural (mantendo os termos da analogia) para reconhecer o Loki e o Gauss, partindo somente das fotos deles.

---

<div style="display: flex; flex-direction: row;">
    <div style="padding: 1rem; background: linear-gradient(to top left, #4B77BE, #89C4F4)">
        <h2>&#127244; Gauss</h2>
        <img src="./imagens_notebook/Gauss.jpg">
    </div>
    <div style="margin-left: 10px; padding: 1rem; background: linear-gradient(to top left, #FFA400, #F4D03F)">
        <h2>&#10102; Loki</h2>
        <img src="./imagens_notebook/Loki.jpg">
    </div>
</div>

Trabalharemos com imagens com tamanho reduzido (120x90 pixels) para acelerar o treinamento. Nenhum tratamento será feito nas imagens além do redimensionamento.

# &#10102; Carregar fotos de treinamento

A pasta `dataset/` contém as fotos que a rede usará para treinamento. Cada foto contém 10800 pixels ($120\times90$), e separaremos os canais de cor (RGB) de cada pixel, totalizando 32400 ($10800\times3$) elementos de entrada em nossa rede. 

In [23]:
import os
import random
from PIL import Image

nomes = [b'Gauss', b'Loki']
# Gauss terá o código 0
# Loki terá o código 1

def carregar_imagens(treinamento = True):
    pasta = "treinamento" if treinamento else "teste"
    caminho = f"./dataset/{pasta}/"
    arquivos_lidos = os.listdir(caminho)
    random.shuffle(arquivos_lidos)  # mistura as imagens lidas
    tensor_de_imagens = []
    matriz_de_codigos = [[]]

    for arquivo in arquivos_lidos:
        foto = Image.open(caminho + arquivo)
        pixels_da_foto = np.array(foto)
        tensor_de_imagens.append(pixels_da_foto)        
        codigo = 0 if "Gauss" in arquivo else 1
        matriz_de_codigos[0].append(codigo)

    return np.array(tensor_de_imagens), np.array(matriz_de_codigos)

tensor_de_imagens_de_treinamento, matriz_de_codigos_de_treinamento = carregar_imagens()
tensor_de_imagens_de_teste, matriz_de_codigos_de_teste = carregar_imagens(treinamento = False)

Ótimo! Agora vamos "desenrolar" a matriz de imagens para servir de entrada em nossa rede neural.

In [24]:
matriz_de_imagens_de_treinamento = tensor_de_imagens_de_treinamento.reshape(tensor_de_imagens_de_treinamento.shape[0], -1).T
matriz_de_imagens_de_teste = tensor_de_imagens_de_teste.reshape(tensor_de_imagens_de_teste.shape[0], -1).T

# Padroniza valores para evitar problemas de instabilidade numérica
matriz_de_treinamento = matriz_de_imagens_de_treinamento/255.
matriz_de_teste = matriz_de_imagens_de_teste/255.

# &#10103; Montar layout da cozinha

O chef precisa montar o layout da cozinha, com as praças (bancadas) e os equipamentos que serão utilizados pelas brigadas (times).

Neste exemplo, a cozinha será formada por:

| Brigada  | Número de cozinheiros | Responsabilidade                                      | Ativação |
| :------: |:---------------------:| :-----------------------------------------------------| :------: |
| 1        | 20                    | Trabalhar nos ingredientes dos pratos de treinamento  | Freezer  |
| 2        | 9                     | Trabalhar no *mise en place* feito pelo primeiro time | Freezer  |
| 3        | 7                     | Trabalhar no *mise en place* feito pelo segundo time  | Freezer  |
| 4        | 5                     | Trabalhar no *mise en place* feito pelo terceiro time | Freezer  |
| 5        | 1                     | Preparar o prato para apresentação ao chef            | Forno    |


In [25]:
# 32400 representa os ingredientes dos pratos de treinamento. 
# Em nosso exemplo, são as intensidades dos pixels das imagens do Loki e Gauss
layout_cozinha = [32400, 20, 9, 7, 5, 1]

# &#10104; Contratar cozinheiros

Ao contratar os cozinheiros, eles chegam com habilidades diversas em relação aos ingredientes que a cozinha trabalha e em relação ao trabalho das brigadas anteriores. Essa função simula isso por iniciar as habilidades de todos de forma aleatória:

In [12]:
import numpy as np

HABILIDADES_BRIGADA = "HABILIDADES_BRIGADA"
EQUIPAMENTOS_BRIGADA = "EQUIPAMENTOS_BRIGADA"

def contrata_time(layout_cozinha):
    np.random.seed(1)
    habilidades_por_brigada = {}
    numero_brigadas = len(layout_cozinha)  # Numero de brigadas na cozinha

    for brigada in range(1, numero_brigadas):
        habilidades_por_brigada[HABILIDADES_BRIGADA + str(brigada)] = np.random.randn(layout_cozinha[brigada], layout_cozinha[brigada - 1]) / np.sqrt(layout_cozinha[brigada - 1]) * 0.01
        habilidades_por_brigada[EQUIPAMENTOS_BRIGADA + str(brigada)] = np.zeros((layout_cozinha[brigada], 1))
        
        assert(habilidades_por_brigada[HABILIDADES_BRIGADA + str(brigada)].shape == (layout_cozinha[brigada], layout_cozinha[brigada-1]))
        assert(habilidades_por_brigada[EQUIPAMENTOS_BRIGADA + str(brigada)].shape == (layout_cozinha[brigada], 1))

        
    return habilidades_por_brigada

Para termos uma noção do que ocorre, vamos analisar as habilidades do time x: #TODO

# &#10105; Descrever atividades das brigadas

> Um primeiro passo nessa atividade é deixar preparado as funções que serão chamadas representando o papel do forno e freezer. Existe uma explicação técnica por trás dessas funções, mas não se preocupe com elas agora.

In [13]:
def forno(resultado_trabalhos_manuais):  # Forno
    mise_en_place = 1/(1 + np.exp(-resultado_trabalhos_manuais))
    caderno_de_ocorrencias = resultado_trabalhos_manuais
    
    return mise_en_place, caderno_de_ocorrencias


def freezer(resultado_trabalhos_manuais):  # Freezer
    mise_en_place = np.maximum(0,resultado_trabalhos_manuais)
    
    assert(mise_en_place.shape == resultado_trabalhos_manuais.shape)
    
    caderno_de_ocorrencias = resultado_trabalhos_manuais
    return mise_en_place, caderno_de_ocorrencias

<div style="background-color: #3498db; display: inline-block; border: 1px solid black; padding: 2px">4.1</div> Primeiro, as brigadas precisam trabalhar nos ingredientes ou no *mise en place* do time anterior. 

In [6]:
def trabalhos_manuais(mise_en_place_anterior, habilidades_da_brigada, equipamentos_da_brigada):
    resultado = habilidades_da_brigada.dot(mise_en_place_anterior) + equipamentos_da_brigada
    
    assert(resultado.shape == (habilidades_da_brigada.shape[0], mise_en_place_anterior.shape[1]))
    caderno_de_ocorrencias = (mise_en_place_anterior, habilidades_da_brigada, equipamentos_da_brigada)
    
    return resultado, caderno_de_ocorrencias

<div style="background-color: #3498db; display: inline-block; border: 1px solid black; padding: 2px">4.2</div> Depois, o resultado do trabalho é organizado para ser utilizado pelo time posterior. Isso é feito levando o que foi preparado para o aparelho disponível pela brigada, um freezer ou forno, produzindo um novo *mise en place*:

In [35]:
def produz_novo_mise_en_place(mise_en_place_anterior, habilidades_da_brigada, equipamentos_da_brigada, aparelho):
    resultado_prep_manual, caderno_de_ocorrencias_manual = trabalhos_manuais(mise_en_place_anterior, habilidades_da_brigada, equipamentos_da_brigada)

    if aparelho == "forno":
        novo_mise_en_place, caderno_de_ocorrencias_aparelho = forno(resultado_prep_manual)    
    elif aparelho == "freezer":
        novo_mise_en_place, caderno_de_ocorrencias_aparelho = freezer(resultado_prep_manual)
    
    assert (novo_mise_en_place.shape == (habilidades_da_brigada.shape[0], mise_en_place_anterior.shape[1]))
    caderno_de_ocorrencias = (caderno_de_ocorrencias_manual, caderno_de_ocorrencias_aparelho)

    return novo_mise_en_place, caderno_de_ocorrencias

<div style="background-color: #3498db; display: inline-block; border: 1px solid black; padding: 2px">4.3</div> Por fim, preparamos uma função responsável por percorrer todas as brigadas, cada uma realizando o seu trabalho e disponibilizando para a brigada posterior, até que a última brigada faça a preparação dos pratos.

In [29]:
def prepara_pratos(ingredientes, habilidades_por_brigada):
    caderno_de_ocorrencias = []
    mise_en_place = ingredientes
    numero_brigadas = len(habilidades_por_brigada) // 2  # número de brigadas na cozinha
    
    for brigada in range(1, numero_brigadas):
        preparacao_time_anterior = mise_en_place 
        mise_en_place, ocorrencias = produz_novo_mise_en_place(preparacao_time_anterior, habilidades_por_brigada[HABILIDADES_BRIGADA + str(brigada)], habilidades_por_brigada[EQUIPAMENTOS_BRIGADA + str(brigada)], aparelho = "freezer")
        caderno_de_ocorrencias.append(ocorrencias)
    
    pratos, ocorrencias = produz_novo_mise_en_place(mise_en_place, habilidades_por_brigada[HABILIDADES_BRIGADA + str(numero_brigadas)], habilidades_por_brigada[EQUIPAMENTOS_BRIGADA + str(numero_brigadas)], aparelho = "forno")
    caderno_de_ocorrencias.append(ocorrencias)
    
    assert(pratos.shape == (1,ingredientes.shape[1]))
            
    return pratos, caderno_de_ocorrencias

# &#10106; Definir como encontrar a culpa das brigadas/aparelhos

As funções abaixo definem o procedimento para encontrar a culpa da habilidade do time, dos equipamentos utilizados pelo time e quando da culpa veio do *mise en place* produzido pelo time anterior:


In [45]:
def acha_culpa_freezer(culpa_do_aparelho_posterior, caderno_de_ocorrencias):    
    preparacao_manual = caderno_de_ocorrencias
    culpa_da_preparacao_manual = np.array(culpa_do_aparelho_posterior, copy=True) 
    
    culpa_da_preparacao_manual[preparacao_manual <= 0] = 0
    
    assert (culpa_da_preparacao_manual.shape == preparacao_manual.shape)
    
    return culpa_da_preparacao_manual


def acha_culpa_forno(culpa_da_montagem, caderno_de_ocorrencias):
    preparacao_manual = caderno_de_ocorrencias
    
    s = 1/(1+np.exp(-preparacao_manual))
    culpa_da_preparacao_manual = culpa_da_montagem * s * (1-s)
    
    assert (culpa_da_preparacao_manual.shape == preparacao_manual.shape)
    
    return culpa_da_preparacao_manual


def acha_culpa_mise_en_place_anterior(culpa_preparacao_manual, caderno_de_ocorrencias):
    mise_en_place_recebido, habilidades_brigada, equipamentos_brigada = caderno_de_ocorrencias
    numero_pratos = mise_en_place_recebido.shape[1]

    culpa_habilidades_brigada = 1./numero_pratos * np.dot(culpa_preparacao_manual, mise_en_place_recebido.T)
    culpa_equipamentos_brigada = 1./numero_pratos * np.sum(culpa_preparacao_manual, axis = 1, keepdims = True)
    culpa_mise_en_place_recebido = np.dot(habilidades_brigada.T, culpa_preparacao_manual)
    
    assert (culpa_mise_en_place_recebido.shape == mise_en_place_recebido.shape)
    assert (culpa_habilidades_brigada.shape == habilidades_brigada.shape)
    assert (culpa_equipamentos_brigada.shape == equipamentos_brigada.shape)
    
    return culpa_mise_en_place_recebido, culpa_habilidades_brigada, culpa_equipamentos_brigada


def acha_culpa_brigada(culpa_aparelho, caderno_de_ocorrencias, aparelho):
    preparacao_manual, mise_en_place = caderno_de_ocorrencias
    
    if aparelho == "freezer":
        culpa_preparacao_manual = acha_culpa_freezer(culpa_aparelho, mise_en_place)
    elif aparelho == "forno":
        culpa_preparacao_manual = acha_culpa_forno(culpa_aparelho, mise_en_place)
    
    culpa_mise_en_place, culpa_habilidades_brigada, culpa_equipamentos_brigada = acha_culpa_mise_en_place_anterior(culpa_preparacao_manual, preparacao_manual)
    
    return culpa_mise_en_place, culpa_habilidades_brigada, culpa_equipamentos_brigada

Ah, finalmente chegamos ao backpropagation. Essa fase é como se fosse uma retrospectiva para o time, para que verifiquem onde erraram e aprendam.

Primeiro, o chef calcula uma pontuação de erros. Quanto maior, pior. A partir daí passa a distribuir a culpa entre todos os membros do time.

In [20]:
def calcula_pontuacao_de_erro(pratos_do_time, pratos_corretos):
    numero_pratos = pratos_corretos.shape[1]

    erro = (1./numero_pratos) * (-np.dot(pratos_corretos,np.log(pratos_do_time).T) - np.dot(1 - pratos_corretos, np.log(1 - pratos_do_time).T))
    
    erro = np.squeeze(erro)
    assert(erro.shape == ())
    
    return erro

In [51]:
CULPA_MISE_EN_PLACE_RECEBIDO = "CULPA_MISE_EN_PLACE_RECEBIDO"
CULPA_HABILIDADES_BRIGADA = "CULPA_HABILIDADES_BRIGADA"
CULPA_EQUIPAMENTOS_BRIGADA = "CULPA_EQUIPAMENTOS_BRIGADA"

def distribui_culpa(pratos_do_time, pratos_corretos, caderno_de_ocorrencias):
    culpa = {}
    numero_brigadas = len(caderno_de_ocorrencias) 
    numero_pratos = pratos_do_time.shape[1]
    pratos_corretos = pratos_corretos.reshape(pratos_do_time.shape) 
    
    erro_na_tentativa = - (np.divide(pratos_corretos, pratos_do_time) - np.divide(1 - pratos_corretos, 1 - pratos_do_time))
    
    ocorrencia_atual = caderno_de_ocorrencias[numero_brigadas - 1]
    culpa[CULPA_MISE_EN_PLACE_RECEBIDO + str(numero_brigadas)], culpa[CULPA_HABILIDADES_BRIGADA + str(numero_brigadas)], culpa[CULPA_EQUIPAMENTOS_BRIGADA + str(numero_brigadas)] = acha_culpa_brigada(erro_na_tentativa, ocorrencia_atual, aparelho = "forno")
    
    for brigada in reversed(range(numero_brigadas - 1)):
        ocorrencia_atual = caderno_de_ocorrencias[brigada]
        culpa_mise_en_place, culpa_habilidades_brigada, culpa_equipamentos_brigada = acha_culpa_brigada(culpa[CULPA_MISE_EN_PLACE_RECEBIDO + str(brigada + 2)], ocorrencia_atual, aparelho = "freezer")
        culpa[CULPA_MISE_EN_PLACE_RECEBIDO + str(brigada + 1)] = culpa_mise_en_place
        culpa[CULPA_HABILIDADES_BRIGADA + str(brigada + 1)] = culpa_habilidades_brigada
        culpa[CULPA_EQUIPAMENTOS_BRIGADA + str(brigada + 1)] = culpa_equipamentos_brigada

    return culpa

# &#10107; Chef faz as melhorias de habilidade e de equipamentos necessárias

...

In [17]:
def upgrade_habilidades_equipamentos(habilidades_por_brigada, culpa, grosseria_do_chef):
    numero_de_brigadas = len(habilidades_por_brigada) // 2 

    for brigada in range(numero_de_brigadas):
        habilidades_por_brigada[HABILIDADES_BRIGADA + str(brigada + 1)] = habilidades_por_brigada[HABILIDADES_BRIGADA + str(brigada + 1)] - grosseria_do_chef * culpa[CULPA_HABILIDADES_BRIGADA + str(brigada + 1)]
        habilidades_por_brigada[EQUIPAMENTOS_BRIGADA + str(brigada + 1)] = habilidades_por_brigada[HABILIDADES_BRIGADA + str(brigada + 1)] - grosseria_do_chef * culpa[CULPA_HABILIDADES_BRIGADA + str(brigada + 1)]
        
    return habilidades_por_brigada

# &#10108; Treinamento

"Olhem aqui, (seus merdinhas|cozinheiros|time)"...

In [53]:
import matplotlib.pyplot as plt

def treinamento(ingredientes, pratos_corretos, layout_cozinha, grosseria = 0.0075, tentativas = 3000):
    np.random.seed(1)
    erros = [] 
    
    habilidades_por_brigada = contrata_time(layout_cozinha)

    for tentativa in range(0, tentativas):
        pratos_do_time, caderno_de_ocorrencias = prepara_pratos(ingredientes, habilidades_por_brigada)
        pontuacao_de_erro = calcula_pontuacao_de_erro(pratos_do_time, pratos_corretos)

        culpa = distribui_culpa(pratos_do_time, pratos_corretos, caderno_de_ocorrencias)

        habilidades_por_brigada = upgrade_habilidades_equipamentos(habilidades_por_brigada, culpa, grosseria)

        if tentativa % 100 == 0:
            print ("Pontuação de erro após %i tentativas: %f" %(tentativa, pontuacao_de_erro))
            erros.append(pontuacao_de_erro)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('erro')
    plt.xlabel('tentativas')
    plt.title("Grosseria = " + str(grosseria))
    plt.show()
    
    return habilidades_por_brigada

Agora vamos ao treinamento...

In [54]:
habilidades_por_brigada = treinamento(matriz_de_imagens_de_treinamento, matriz_de_codigos_de_treinamento, layout_cozinha, tentativas = 2500)

Pontuação de erro após 0 tentativas: 0.693147


ValueError: operands could not be broadcast together with shapes (20,123) (20,32400) 